Load libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from fastai.vision import *

Look at dataset

In [ ]:
path = Path('../input')

In [ ]:
path.ls()

In [ ]:
path_img = path/'train'

In [ ]:
get_image_files(path_img)[:5]

Create datablock

In [ ]:
np.random.seed(42)
size = 224
bs = 64
num_workers = 0  # set this to 0 to prevent kernel from crashing
pat = r'/([^/.]+).\d+.jpg$'

In [ ]:
tfms = get_transforms()                              #Do standard data augmentation
data = (ImageItemList.from_folder(path_img)          #Get the training images from the train dir
        .random_split_by_pct()                       #Randomly split off 20% of the images to form validation set
        .label_from_re(pat)                          #Label by applying the regex to the filenames
        .add_test_folder('../test')                  #Add a test set using the test dir
        .transform(tfms, size=size)                  #Pass in data augmentation
        .databunch(bs=bs, num_workers=num_workers)   #Create ImageDataBunch
        .normalize(imagenet_stats))                  #Normalize using imagenet stats

Verify datasets loaded properly.  We should have the following:
* train: 20,000
* valid: 5,000
* test: 12,500

In [ ]:
print(len(data.train_ds))
print(len(data.valid_ds))
print(len(data.test_ds))

In [ ]:
data.classes

Visualize data

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

Create learner using pre-trained model

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=accuracy, model_dir='/tmp/models')

Fit the model with most layers frozen

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

Unfreeze all layers, run learning rate finder, fit some more

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('stage-2')

Examine prediction errors in validation set

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(6,6), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

Create predictions using test set

In [ ]:
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
dog_preds = preds[:,1]

Generate competition submission using predictions

In [ ]:
submission = pd.DataFrame({'id':os.listdir('../input/test'), 'label':dog_preds})

In [ ]:
submission['id'] = submission['id'].map(lambda x: x.split('.')[0])

In [ ]:
submission['id'] = submission['id'].astype(int)

In [ ]:
submission = submission.sort_values('id')

In [ ]:
submission.to_csv('submission.csv', index=False)